# Result-Dependent if/else

task result dependent if/else statements inside lattices (B)

In case you'd like to use some conditional logic in your workflow, say, something like method (1), it'll be better if you introduced it in the resulting electron directly instead of inside the lattice, something like method (2). The reason for that is that during the workflow dependency graph creation, the return values of the `task` functions is unknown, thus these results are of the type `Electron`. This is because the execution doesn't happen, and thus the dependency of logic on the result of the task cannot be relied upon.

To branch based on the result from an electron, put the branch (if/else) logic inside another electron.

## Context

Often the output of one task is a value used to choose the execution path. In these cases, perform the if/else logic inside an electron. 

When a lattice is dispatched, the Covalent server executes the lattice in order to build the transport graph. The transport graph is then analyzed to parallelize electron execution on their assigned executors.

If the server encounters a branch decision based on the output of an electron, it cannot infer the structure on which the decision depends and is prevented from building the transport graph.

## Best Practice

Compute branching inside an electron. Electrons execution is deferred during the graph build phase, so their output cannot be used to build the transport graph and analyze the execution for parallelization.

**Exception**: Sublattices are evaluated during lattice execution and their graph structure is "plugged in" to the larger transport graph. If a sublattice produces a static decision variable (does not produce it by executing an electron internally), then it is safe to branch on.

## Example

Contrast the two examples below.

### Not Recommended

This example demonstrates a questionable approach: choosing the execution path in the lattice based on the output of `task_1`.

In [ ]:
import covalent as ct

# Technique 1:

@ct.electron
def task_1(x):
    return x * 2

@ct.electron
def task_2(x):
    return x ** 3

@ct.lattice
def workflow(a):
    
    res_1 = task_1(a)

    if res_1 == 10:
        final_res = task_2(res_1)
    else:
        final_res = res_1
    
    return final_res

### Improved

The output of `task_1` is passed to the `task_2_new`, which executes the chosen path internally and returns the result.

In [ ]:
import covalent as ct

# Technique 2:

@ct.electron
def task_1(x):
    return x * 2

# Method (2):
@ct.electron
def task_2_new(x):
    if x == 10:
        return x ** 3
    else:
        return x

@ct.lattice
def workflow_2(a):
    res_1 = task_1(a)
    return task_2_new(res_1)

## Keywords

- Branch
- if/else
- Execution path

## Benefits

Typical benefits that might appear here:
- Performance
- Simplicity
- Maintainability
- Resource (CPU, memory) conservation
- Efficiency
- Workaround (to a problem with Python or with Covalent)

## See Also

Other Best Practices or How-to. Other documentation (Concepts, API Reference) if especially applicable.
